## SICP 习题 （3.16）解题总结: 序对的统计

SICP 习题 3.16 通过一个错误的序对统计函数讲解car, cdr的具体工作方式。

我们在完成SICP习题的时候要时刻留意题目的真正目的。像这道题，目的根本就不是为了计算序对的个数，而是了解序对的工作机制，car,cdr的作用，还有因为set-car! set-cdr!可能导致的复杂情况，如环链。

开始细看题目之前稍微重温一下序对的概念，序对就是通过cons创建出来的形如(a . b)的东西，一个列表看起来是(1 2 3)这个样子的，其实它在本质上是(1 . (2 . (3 . ())))。

题目说Ben Bitdiddle决定写一个统计序对的函数，代码如下：

In [9]:
(define (count-pairs x)
  (if (not (pair? x))
      0
      (+ (count-pairs (car x))
         (count-pairs (cdr x))
         1)))

代码的逻辑初看是没有问题的，我们可以把一个列表当一颗树来看待，分别看看左树有多少个元素，右树有多少个元素，再加上当前这个元素，就是整个列表的元素个数。

这个代码对普通的列表是没有问题的，测试一下：

In [10]:
(count-pairs '(1 2 3))

3

注意，对下面这种形式的列表，它也是正常工作的，虽然我们看到3个元素，但是事实上里面有4个序对.

列表(1 2 (3))在本质上是 (1 . (2 . ((3 . ()) . ())))

In [20]:
(count-pairs '(1 2 (3)))

4

但是，如果列表中有序对指向同一个元素，那么就出问题了，看看下面这个结果：

In [18]:
(define leaf (cons 1 2))
(define to-same-leaf (cons leaf leaf))
(define test-sample (cons to-same-leaf 3))

In [19]:
(count-pairs test-sample)

4

为什么会返回4呢，因为to-same-leaf的car和cdr同时指向了leaf，Ben设计的代码会重复计算leaf这个序对


按这个逻辑，我们还可以通过下面的代码统计出5个元素来：

In [23]:
(define leaf (cons 1 2))
(define to-same-leaf (cons leaf leaf))
(define test-sample (cons to-same-leaf leaf))
(count-pairs test-sample)

5

题目还要求我们找出样例是可以算出7个的，如果理解了Ben的函数的问题所在，就知道我们需要尽可能地重复指向相同的元素，可以是这样：

In [25]:
(define leaf (cons 1 2))
(define to-same-leaf (cons leaf leaf))
(define test-sample (cons to-same-leaf to-same-leaf))
(count-pairs test-sample)

7

题目还提示说不定Ben这个函数不会返回，什么情况下不会返回呢？前面练习里提到的那种环链就不会返回，可以统计出无限个序对出来。。。。。